# TP6

### `Resolver usando Pandas`

Resolver los ejercicios del TP3 utilizando la librería Pandas.

### Ejercicio 1: Cargar Datos de ventas.

El archivo datos.dat contiene el registro de las ventas realizadas.

Tiene un formato de ancho fijo:
- `fecha`:    10 lugares
- `producto`: 30 lugares
- `precio`:   10 lugares
- `cantidad`:  5 lugares

Hacer una funcion que cargue los datos en un DataFrame de Pandas.

In [1]:
import pandas as pd

def cargar_datos():
    column_names = ['fecha', 'producto', 'precio', 'cantidad']
    
    colspecs = [(0, 10), (10, 40), (40, 50), (50, 55)] 

    datos = pd.read_fwf('datos.dat', colspecs=colspecs, header=None, names=column_names)
    
    datos['fecha'] = pd.to_datetime(datos['fecha'].str.strip())  
    datos['producto'] = datos['producto'].str.strip()  
    datos['precio'] = datos['precio'].astype(float)  
    datos['cantidad'] = datos['cantidad'].astype(int)  
    
    return datos

datos = cargar_datos()
print(datos)


        fecha    producto  precio  cantidad
0  2024-08-27     Mirinda  1510.0        14
1  2024-08-27     Mirinda  1560.0        12
2  2024-08-28     Torasso   940.0         8
3  2024-08-29  Pepsi Cola  1210.0        10
4  2024-08-30     Mirinda  1520.0         1
5  2024-09-01     Mirinda  1550.0        15
6  2024-09-01      Sprite   810.0         4
7  2024-09-02   Coca Cola  1100.0         4
8  2024-09-02  Pepsi Cola  1220.0        13
9  2024-09-02     Torasso   910.0         5
10 2024-09-02     Torasso   920.0         3
11 2024-09-03   Coca Cola  1020.0         8
12 2024-09-03     Mirinda  1570.0         7
13 2024-09-03     Mirinda  1590.0         2
14 2024-09-04  Pepsi Cola  1220.0        13
15 2024-09-05     Mirinda  1500.0         3
16 2024-09-05  Pepsi Cola  1300.0         5
17 2024-09-06   Coca Cola  1080.0         1
18 2024-09-06      Sprite   860.0        12
19 2024-09-06     Torasso   930.0         3
20 2024-09-07   Coca Cola  1080.0        14
21 2024-09-07      Sprite   870.

### Ejercicio 2: Calcular el total de ventas.
Hacer una función que sume los importes vendidos (precio * cantidad) y las cantidades.


In [2]:
def calcular_totales(datos):
    datos['importe'] = datos['precio'] * datos['cantidad']
    
    total_importe = datos['importe'].sum()
    total_cantidad = datos['cantidad'].sum()
    
    return total_importe, total_cantidad

importe, cantidad = calcular_totales(datos)

print(f"Las ventas fueron de ${importe:.2f} en {cantidad} unidades")


Las ventas fueron de $392730.00 en 335 unidades


### Ejercicio 3: Listar las unidades vendidas.
Listar cuántas unidades se vendieron en total para cada producto


In [3]:
def unidades_vendidas(datos):
    resumen = datos.groupby('producto')['cantidad'].sum().reset_index()
    resumen.columns = ['producto', 'total_unidades']
    
    return resumen

unidades_totales = unidades_vendidas(datos)

print(unidades_totales)


     producto  total_unidades
0   Coca Cola              57
1     Mirinda              85
2  Pepsi Cola              89
3      Sprite              72
4     Torasso              32


###  Ejercicio 4: Listar el precio promedio por producto.
Hacer un listado del precio promedio por producto.


In [4]:
def precio_promedio(datos):
    # Agrupar por producto y calcular el precio promedio
    resumen = datos.groupby('producto')['precio'].mean().reset_index()
    resumen.columns = ['producto', 'precio_promedio']
    
    return resumen

precios_promedio = precio_promedio(datos)

print(precios_promedio)


     producto  precio_promedio
0   Coca Cola      1072.500000
1     Mirinda      1545.833333
2  Pepsi Cola      1245.000000
3      Sprite       841.428571
4     Torasso       920.000000


### Ejercicio 5: Ranking de productos
Realizar un listado de los 3 productos más vendidos ordenados por la cantidad de unidades vendidas (ordenadas de mayor a menor)


In [5]:
def ranking_productos(datos, top=3):
    ranking = datos.groupby('producto')['cantidad'].sum().reset_index()
    
    ranking.columns = ['producto', 'total_unidades']
    
    ranking = ranking.sort_values(by='total_unidades', ascending=False)
    
    return ranking.head(top)

top_productos = ranking_productos(datos)

print(top_productos)


     producto  total_unidades
2  Pepsi Cola              89
1     Mirinda              85
3      Sprite              72


### Ejercicio 6: Listar las ventas por mes
Realizar un listado del total de unidades vendidas por producto separado por mes.


In [6]:
def ventas_por_mes(datos):
    # Añadir una columna de mes para agrupar
    datos['mes'] = datos['fecha'].dt.to_period('M')
    
    resumen = datos.groupby(['producto', 'mes'])['cantidad'].sum().reset_index()
    
    resumen.columns = ['producto', 'mes', 'total_unidades']
    
    return resumen

ventas_mensuales = ventas_por_mes(datos)

print(ventas_mensuales)


     producto      mes  total_unidades
0   Coca Cola  2024-09              57
1     Mirinda  2024-08              27
2     Mirinda  2024-09              58
3  Pepsi Cola  2024-08              10
4  Pepsi Cola  2024-09              79
5      Sprite  2024-09              72
6     Torasso  2024-08               8
7     Torasso  2024-09              24


### Ejercicio 7: Informe general

Mostrar un listado de productos ordenados alfabeticamente que contengan el precio promedio, la cantidad de unidades vendidas y el importe total vendido para cada producto

In [7]:
def resumen_ventas(datos):
    precios_promedio = datos.groupby('producto')['precio'].mean().reset_index()
    precios_promedio.columns = ['producto', 'precio_promedio']
    
    total_unidades = datos.groupby('producto')['cantidad'].sum().reset_index()
    total_unidades.columns = ['producto', 'total_unidades']
    
    datos['importe'] = datos['precio'] * datos['cantidad']  
    total_importe = datos.groupby('producto')['importe'].sum().reset_index()
    total_importe.columns = ['producto', 'importe_total']
    
    resumen = pd.merge(precios_promedio, total_unidades, on='producto')
    resumen = pd.merge(resumen, total_importe, on='producto')
    
    resumen = resumen.sort_values(by='producto').reset_index(drop=True)
    
    return resumen

informe_general = resumen_ventas(datos)

print(informe_general)


     producto  precio_promedio  total_unidades  importe_total
0   Coca Cola      1072.500000              57        60780.0
1     Mirinda      1545.833333              85       131080.0
2  Pepsi Cola      1245.000000              89       110510.0
3      Sprite       841.428571              72        61040.0
4     Torasso       920.000000              32        29320.0


## `Resolver usando NumPy`
## Resolver el ejercicio 2 del tp1 usando NumPy

### Ejercicio 8

Escribe una función en Python que encuentre los valores de `a`, `b`, y `c` para que la función cuadrática `f(x) = a x^2 + b x + c` pase exactamente por los siguientes puntos:

| x  | y  |
|---:|---:|
|  0 |  0 |
|  1 |  8 |
|  2 | 12 |
|  3 | 12 |
|  5 |  0 |

### Requisitos:
- La función debe explorar posibles valores de `a`, `b`, y `c` utilizando un método de prueba y error.
- Debe devolver los valores que hagan que la diferencia entre la función `f(x)` y los valores medidos `y` sea exactamente cero para cada punto.

> **Pista**: Los valores de `a`, `b`, y `c` son números pequeños.

La idea es implementar el mismo algoritmo que se uso en el TP1 pero usando NumPy en lugar de Python puro.

In [12]:
import numpy as np

def f(x, coeficientes):
    a, b, c = coeficientes
    return a * x**2 + b * x + c

def buscar_coeficientes():
    valores_a = np.arange(-10, 11, 1)  
    valores_b = np.arange(-10, 11, 1)  
    valores_c = np.arange(-10, 11, 1) 
    
    coeficientes_validos = []

    X = np.array([0, 1, 2, 3, 5])
    Y = np.array([0, 8, 12, 12, 0])
    
    for a in valores_a:
        for b in valores_b:
            for c in valores_c:
                Y_pred = f(X, (a, b, c))
                
                if np.all(Y_pred == Y): 
                    coeficientes_validos.append((int(a), int(b), int(c)))
    
    return coeficientes_validos

coeficientes = buscar_coeficientes()
if coeficientes:
    resultado = coeficientes[0]
    print(resultado) 


(-2, 10, 0)


### Ejercicio 9: Resolver el ejercicio 3 del TP1 usando NumPy
Buscar los coeficientes de la función que minimice la suma de los cuadrados de las diferencias entre los valores medidos y los valores de la función.

1. Crear un array con los coeficientes elegidos al azar (usar `randint(-10,10,3)`).
2. Calcular el valor de la función y el error correspondiente.
3. Mientras que el error sea mayor a 1:
    1. Definir nuevos coeficientes agregándoles un pequeño valor al azar a los coeficientes actuales (aprendizaje = 0.001).
    2. Si el error para los nuevos coeficientes es menor que el anterior, reemplazar los coeficientes actuales por los nuevos.


In [11]:
import numpy as np
from numpy.random import randint

def f(x, coeficientes):
    a, b, c = coeficientes
    return a * x**2 + b * x + c

def error(y, y_pred):
    return np.sum((y - y_pred) ** 2)

def buscar_coeficientes():
    coeficientes = randint(-10, 10, 3)
    learning_rate = 0.001
    
    Y_pred = f(X, coeficientes)
    current_error = error(Y, Y_pred)

    while current_error > 1: 
        nuevos_coeficientes = coeficientes + np.random.uniform(-learning_rate, learning_rate, 3)
        
        Y_pred_nuevos = f(X, nuevos_coeficientes)
        nuevo_error = error(Y, Y_pred_nuevos)
        
        if nuevo_error < current_error:
            coeficientes = nuevos_coeficientes
            current_error = nuevo_error
            
    return coeficientes

X = np.array([0, 1, 2, 3, 5])
Y = np.array([0, 8, 12, 11, 1])  

coeficientes = buscar_coeficientes()
print(coeficientes)


[-1.78013311  8.96030852  0.62367986]
